# Feature Extraction
## Importing Libraries

In [1]:
# import cv2 as cv
import SimpleITK as sitk

import numpy as np
import os
from skimage.feature.texture import greycomatrix
from skimage.feature.texture import greycoprops
from skimage.measure import shannon_entropy
from radiomics import glrlm, glcm
# import pyfeats\\
import pandas as pd
import multiprocessing as mlp
import math
import feature_extraction as fe

In [2]:
# from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

## Define Feature Extraction functions

### Read dataset images

In [3]:
def read_images(folder = "dataset/train",
                classes = [
                            "normal",
                            "fatty",
                            "cirrhosis"
                        ]):
    image_names = {}
    images = []
    # Get all image names in folders
    for cls in classes:
        image_names[cls] = os.listdir(f'{folder}/{cls}')

    # read all images to list
    for cls in classes:
        for name in image_names[cls]:
            mask = []
            with open(f'dataset/masks/{name[0:-4]}.txt', 'r') as file:
                data = file.read()
                data = data.strip().split('\n')
                for line in data:
                    x, y = line.split(',')
                    mask.append((int(y),int(x)))
            img = sitk.ReadImage(f'{folder}/{cls}/{name}', sitk.sitkUInt8)
            images.append((name, img,cls,mask))
    return images

### Extract ROIs from image

In [4]:
def extract_roi(img, start , size = (32,32)):
    img = sitk.GetArrayFromImage(img)
    roi = img[start[0]:start[0]+size[0],start[1]:start[1]+size[1]]
    mask = np.zeros(img.shape)
    mask[start[0]:start[0]+size[0],start[1]:start[1]+size[1]] = 1
    return roi, mask

# Calculate Liver Diagonal Length

In [5]:
def get_length(img, mask):
    # top right, bottom left
    tr_distance = []
    bl_distance = []

    # top left, bottom right
    tl_distance = []
    br_distance = []


    for x,y in mask:
        tr_distance.append(math.dist([0,img.shape[1]],[x+32,y]))
        bl_distance.append(math.dist([img.shape[0],0],[x,y+32]))

        tl_distance.append(math.dist([0,0],[x,y]))
        br_distance.append(math.dist(img.shape,[x+32,y+32]))


    top_right = mask[tr_distance.index(min(tr_distance))]
    bottom_left = mask[bl_distance.index(min(bl_distance))]

    top_left = mask[tl_distance.index(min(tl_distance))]
    bottom_right = mask[br_distance.index(min(br_distance))]
    
    return max(math.dist(top_right,bottom_left),math.dist(top_left,bottom_right))

### Extract Features from ROIs

In [6]:
def feature_extraction(img, roi_pos):
    roi_mask_arr = []
    for pos in roi_pos:
        roi_mask_arr.append(extract_roi(img, pos))

    # 0 45 90 135 degrees
    angles = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]

    da_dict = {
        0: "d1_0",1: "d1_45",2: "d1_90",3: "d1_135",
        4: "d2_0",5: "d2_45",6: "d2_90",7: "d2_135",
        8: "d3_0",9: "d3_45",10: "d3_90",11: "d3_135"
    }

    length = get_length(sitk.GetArrayFromImage(img), roi_pos)

    feat_arr = []
    for roi, mask in roi_mask_arr:
        features = {}

        glcm_mtx = greycomatrix(roi, distances = [1,2,3], angles = angles, levels = 256)
        con = greycoprops(glcm_mtx, 'contrast').flatten()
        hom = greycoprops(glcm_mtx, 'homogeneity').flatten()
        en = greycoprops(glcm_mtx, 'energy').flatten()
        corr = greycoprops(glcm_mtx, 'correlation').flatten()

        for j in range(len(da_dict)):
            features[f'contrast_{da_dict[j]}'] = con[j]
            features[f'homogeneity_{da_dict[j]}'] = hom[j]
            features[f'energy_{da_dict[j]}'] = en[j]
            features[f'correlation_{da_dict[j]}'] = corr[j]

        features[f'entropy'] = shannon_entropy(roi)

        features['length'] = length

        # pyradiomics
        mask = sitk.GetImageFromArray(mask)
        # First Order features
        firstOrderFeatures = firstorder.RadiomicsFirstOrder(img, mask)
        # firstOrderFeatures.enableFeatureByName('Mean', True)
        firstOrderFeatures.enableAllFeatures()
        results = firstOrderFeatures.execute()
        for col in results.keys():
            features[col] = results[col].item()

        # GLCM features
        glcmFeatures = glcm.RadiomicsGLCM(img, mask)
        glcmFeatures.enableAllFeatures()
        results = glcmFeatures.execute()
        for col in results.keys():
            features[col] = results[col].item()
        #
        # GLRLM features
        glrlmFeatures = glrlm.RadiomicsGLRLM(img, mask)
        glrlmFeatures.enableAllFeatures()
        results = glrlmFeatures.execute()
        for col in results.keys():
            features[col] = results[col].item()

        feat_arr.append(features)

    return feat_arr

### Construct dataframe from ROI features

In [7]:
def build_dataframe(images):
    # dataframe consists of features of 1 ROI per image
    # column name roiNum_feature
    data = pd.DataFrame()

    for img, cls, mask in images:
        feat_arr = feature_extraction(img, roi_pos=mask)
        for row in feat_arr:
            row['target'] = cls
            data = data.append(row,ignore_index=True)
    return data

### Construct dataframe using multiprocessing
### Reduced runtime by 82%

In [8]:
def build_with_mlp(images, n=9): 
    pool = mlp.Pool(n)
    results = pool.map(fe.build_dataframe,np.array_split(images,n))
    return results

## Feature Analysis and Selection

### Extract Features and build dataframe

In [9]:
%%time

# images = read_images('dataset/train')
# mlp_data = build_with_mlp(images)
# data = pd.DataFrame()
# for frame in mlp_data:
#     data = data.append(frame)

# data.set_index('name', drop=True, inplace=True)

# data.to_csv("dataset/train.csv")

data = pd.read_excel('dataset/segment/train.xlsx', index_col='name')

data.describe()

Wall time: 19.2 s


,10Percentile,90Percentile,Autocorrelation,ClusterProminence,ClusterShade,ClusterTendency,Contrast,Correlation,DifferenceAverage,DifferenceEntropy,...,homogeneity_d1_90,homogeneity_d2_0,homogeneity_d2_135,homogeneity_d2_45,homogeneity_d2_90,homogeneity_d3_0,homogeneity_d3_135,homogeneity_d3_45,homogeneity_d3_90,length
count,10239.000000,10239.000000,10239.000000,10239.000000,10239.000000,10239.000000,10239.000000,10239.000000,10239.000000,1.023900e+04,...,10239.000000,10239.000000,10239.000000,10239.000000,10239.000000,10239.000000,10239.000000,10239.000000,10239.000000,10239.000000
mean,43.214386,70.884881,5.190461,27.420777,2.159861,1.169789,0.316116,0.520472,0.246282,7.505986e-01,...,0.177287,0.197639,0.162198,0.192916,0.140118,0.162515,0.137177,0.146077,0.135427,294.342970
std,25.953822,33.997997,4.488373,176.498185,12.593087,1.654653,0.331033,0.171877,0.128860,2.695262e-01,...,0.109722,0.113970,0.104919,0.118423,0.101432,0.106142,0.100996,0.104579,0.101120,72.108890
min,0.000000,0.000000,1.000000,0.000000,-20.744314,0.000000,0.000000,-0.003308,0.000000,-3.203427e-16,...,0.041775,0.050039,0.042185,0.043133,0.034135,0.039906,0.035083,0.034000,0.027464,135.764502
25%,24.000000,46.000000,2.729294,0.870617,-0.024615,0.534995,0.170213,0.407430,0.163213,6.255283e-01,...,0.117785,0.130055,0.109110,0.122657,0.093496,0.106743,0.090946,0.094952,0.089072,243.704739
50%,41.000000,69.000000,4.182663,2.084719,0.282485,0.807595,0.254504,0.513868,0.238976,7.787500e-01,...,0.154595,0.173784,0.141294,0.165697,0.117997,0.140509,0.115081,0.122553,0.112606,286.216701
75%,60.000000,91.000000,6.122207,9.357921,0.876473,1.345874,0.373407,0.625236,0.323690,9.169306e-01,...,0.197679,0.226263,0.178031,0.223166,0.151147,0.180674,0.147460,0.161069,0.146184,340.164666
max,174.000000,253.000000,66.896373,5954.836767,390.534435,39.317592,10.517470,1.000000,1.587563,1.962372e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,480.000000


In [10]:
%%time

# test_images = read_images("dataset/test")
# mlp_data = build_with_mlp(test_images)
# test_data = pd.DataFrame()
# for frame in mlp_data:
#     test_data = test_data.append(frame)
    
# test_data.set_index('name', drop=True, inplace=True)

# test_data.to_csv("dataset/test.csv")

test_data = pd.read_excel('dataset/segment/test.xlsx', index_col='name')

test_data.describe()

Wall time: 3.35 s


,10Percentile,90Percentile,Autocorrelation,ClusterProminence,ClusterShade,ClusterTendency,Contrast,Correlation,DifferenceAverage,DifferenceEntropy,...,homogeneity_d1_90,homogeneity_d2_0,homogeneity_d2_135,homogeneity_d2_45,homogeneity_d2_90,homogeneity_d3_0,homogeneity_d3_135,homogeneity_d3_45,homogeneity_d3_90,length
count,2543.000000,2543.000000,2543.000000,2543.000000,2543.000000,2543.000000,2543.000000,2543.000000,2543.000000,2.543000e+03,...,2543.000000,2543.000000,2543.000000,2543.000000,2543.000000,2543.000000,2543.000000,2543.000000,2543.000000,2543.000000
mean,42.901219,71.010814,5.015083,26.339400,2.002938,1.167737,0.314198,0.516472,0.250870,7.536736e-01,...,0.176278,0.197346,0.161127,0.192916,0.140383,0.162683,0.138232,0.146238,0.136276,293.332709
std,26.602086,35.537199,4.006853,201.157591,13.729393,1.703052,0.340934,0.170138,0.138701,2.863724e-01,...,0.114381,0.116635,0.109106,0.123798,0.106084,0.108893,0.105759,0.109164,0.105858,66.869642
min,0.000000,0.000000,1.000000,0.000000,-13.731914,0.000000,0.000000,-0.003336,0.000000,-3.203427e-16,...,0.046256,0.049470,0.038131,0.047311,0.034038,0.039583,0.034590,0.035135,0.032659,135.764502
25%,24.000000,45.000000,2.542497,0.861262,-0.008815,0.524250,0.168639,0.408831,0.161579,6.248237e-01,...,0.116269,0.130361,0.108888,0.121239,0.091311,0.107111,0.090186,0.093158,0.087530,243.704739
50%,41.000000,68.000000,4.091035,1.934584,0.275089,0.815074,0.251024,0.504155,0.241261,7.808226e-01,...,0.151490,0.172473,0.137961,0.161490,0.116903,0.139648,0.114804,0.121216,0.113159,295.025423
75%,62.000000,93.000000,6.060370,8.418262,0.766716,1.332241,0.380118,0.618500,0.331848,9.225272e-01,...,0.193826,0.228049,0.175607,0.219497,0.150460,0.182794,0.149007,0.159531,0.147083,329.460165
max,135.000000,253.000000,40.840677,5220.689962,333.676485,31.207490,9.178053,1.000000,1.135032,1.526040e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,435.247056


## Testing

In [11]:
def split(data, test_data, drop=None, cols = None):
    X_train = data.copy()
    y_train = X_train.pop('target')
    X_test = test_data.copy()
    y_test = X_test.pop('target')

    if drop != None:
        X_train = X_train[y_train != drop]
        X_test = X_test[y_test != drop]

        y_train = y_train[y_train != drop]
        y_test = y_test[y_test != drop]
    
    if cols is None: cols = X_train.columns
    
    std = StandardScaler()
    std.fit(X_train[cols])
    X_train = pd.DataFrame(std.transform(X_train[cols]), columns = cols, index = X_train.index)
    X_test = pd.DataFrame(std.transform(X_test[cols]), columns = cols, index = X_test.index)
    return X_train, y_train, X_test, y_test, std

In [12]:
def train_test(model, X_train, y_train, X_test, y_test):
    model = model.fit(X_train, y_train)
    y_pred = pd.Series(model.predict(X_test),index=y_test.index)
    return model, y_pred

In [13]:
def images_pred(y_pred):
    count = 0
    prediction = {}
    for name in np.unique(y_pred.index):
        pred_cls = {}
        for i in y_pred[name]:
            if i not in pred_cls.keys():
                pred_cls[i]=1
            else: pred_cls[i]+=1
        
        prediction[name] = max(pred_cls, key=pred_cls.get)
    return prediction

In [14]:
def images_acc(y_test, y_pred):
    pred_count = 0
    for key in y_pred.keys():
        if y_test[key][0] == y_pred[key]:
            pred_count += 1
    return pred_count/len(y_pred.keys())

In [15]:
models = {
    "RFC": RandomForestClassifier(
                    random_state=42,
                    max_features='auto',
                    n_estimators= 500,
                    max_depth=6,
                    criterion='entropy'),
    "MLP": MLPClassifier(
                    max_iter=600,
                    momentum=0.6,
                    solver='adam',
                    activation='relu',
                    learning_rate_init=0.005,
                    alpha=0.001,
                    random_state=42),
    "SVC": svm.SVC(random_state=42)
}
classes = ['normal', 'fatty', 'cirrhosis']

for drop in classes:
    X_train, y_train, X_test, y_test, std = split(data, test_data, drop)
    print(*[cls for cls in classes if cls != drop])
    for name in models.keys():
        model, y_pred = train_test(models[name], X_train, y_train, X_test, y_test)
        prediction = images_pred(y_pred)
        print(name," Image Accuracy: ", images_acc(y_test, prediction))
        report = classification_report(y_test, y_pred, output_dict = True)
        cr = pd.DataFrame(report).transpose()
        print(cr)
    print('\n\n')

fatty cirrhosis
RFC  Image Accuracy:  0.8
              precision    recall  f1-score     support
cirrhosis      0.731707  0.066667  0.122200   450.00000
fatty          0.788520  0.993025  0.879035  1577.00000
accuracy       0.787370  0.787370  0.787370     0.78737
macro avg      0.760113  0.529846  0.500617  2027.00000
weighted avg   0.775907  0.787370  0.711015  2027.00000
MLP  Image Accuracy:  0.8461538461538461
              precision    recall  f1-score      support
cirrhosis      0.550595  0.411111  0.470738   450.000000
fatty          0.843288  0.904249  0.872705  1577.000000
accuracy       0.794771  0.794771  0.794771     0.794771
macro avg      0.696942  0.657680  0.671721  2027.000000
weighted avg   0.778309  0.794771  0.783467  2027.000000
SVC  Image Accuracy:  0.8
              precision    recall  f1-score      support
cirrhosis      0.714286  0.066667  0.121951   450.000000
fatty          0.788413  0.992391  0.878720  1577.000000
accuracy       0.786877  0.786877  0.78687

In [29]:
# feat importance
files = ['fatty_normal', 'cirrhosis_fatty', 'cirrhosis_normal']
features_acc={}
for name in files:
    features_acc[name] = pd.read_csv(f'dataset/segment/manual selection/{name}.csv', index_col = 0)

In [30]:
normal_fatty_mlp = MLPClassifier(
                    max_iter=600,
                    momentum=0.6,
                    solver='adam',
                    activation='relu',
                    learning_rate_init=0.005,
                    alpha=0.001,
                    random_state=31
                    )

feat_imp = features_acc['fatty_normal']['ANN Accuracy'].sort_values(ascending=False)
normal_fatty_cols = feat_imp.index[0:19]

X_train, y_train, X_test, y_test, normal_fatty_std = split(data, test_data, 'cirrhosis', cols = normal_fatty_cols)
model, y_pred = train_test(normal_fatty_mlp, X_train, y_train, X_test, y_test)
normal_fatty_mlp = model
prediction = images_pred(y_pred)
print("Normal/Fatty MLP Image Accuracy: ", images_acc(y_test, prediction))
report = classification_report(y_test, y_pred, output_dict = True)
cr = pd.DataFrame(report).transpose()
print(cr)

Normal/Fatty MLP Image Accuracy:  0.8059701492537313
              precision    recall  f1-score      support
fatty          0.800961  0.951807  0.869893  1577.000000
normal         0.652968  0.277132  0.389116   516.000000
accuracy       0.785475  0.785475  0.785475     0.785475
macro avg      0.726964  0.614470  0.629504  2093.000000
weighted avg   0.764475  0.785475  0.751364  2093.000000


In [31]:
normal_cirrhosis_mlp = MLPClassifier(
                    max_iter=600,
                    momentum=0.6,
                    solver='adam',
                    activation='relu',
                    learning_rate_init=0.005,
                    alpha=0.001,
                    random_state=81
                    )

feat_imp = features_acc['cirrhosis_normal']['ANN Accuracy'].sort_values(ascending=False)
normal_cirrhosis_cols = feat_imp.index[0:21].insert(0,'length')
X_train, y_train, X_test, y_test, normal_cirrhosis_std = split(data, test_data, 'fatty', cols = normal_cirrhosis_cols)
model, y_pred = train_test(normal_cirrhosis_mlp, X_train, y_train, X_test, y_test)
normal_cirrhosis_mlp = model
prediction = images_pred(y_pred)
print("normal/cirrhosis MLP Image Accuracy: ", images_acc(y_test, prediction))
report = classification_report(y_test, y_pred, output_dict = True)
cr = pd.DataFrame(report).transpose()
print(cr)

normal/cirrhosis MLP Image Accuracy:  0.6071428571428571
              precision    recall  f1-score     support
cirrhosis      0.604423  0.546667  0.574096  450.000000
normal         0.635063  0.687984  0.660465  516.000000
accuracy       0.622153  0.622153  0.622153    0.622153
macro avg      0.619743  0.617326  0.617280  966.000000
weighted avg   0.620789  0.622153  0.620231  966.000000


In [32]:
fatty_cirrhosis_mlp = MLPClassifier(
                    max_iter=600,
                    momentum=0.6,
                    solver='adam',
                    activation='relu',
                    learning_rate_init=0.005,
                    alpha=0.001,
                    random_state=53
                    )

feat_imp = features_acc['cirrhosis_fatty']['ANN Accuracy'].sort_values(ascending=False)
fatty_cirrhosis_cols = feat_imp.index[0:63].insert(0,'length')

X_train, y_train, X_test, y_test, fatty_cirrhosis_std = split(data, test_data, 'normal', cols = fatty_cirrhosis_cols)
model, y_pred = train_test(fatty_cirrhosis_mlp, X_train, y_train, X_test, y_test)
fatty_cirrhosis_mlp = model
prediction = images_pred(y_pred)
print("cirrhosis/Fatty MLP Image Accuracy: ", images_acc(y_test, prediction))
report = classification_report(y_test, y_pred, output_dict = True)
cr = pd.DataFrame(report).transpose()
print(cr)

cirrhosis/Fatty MLP Image Accuracy:  0.8153846153846154
              precision    recall  f1-score      support
cirrhosis      0.535065  0.457778  0.493413   450.000000
fatty          0.851401  0.886493  0.868593  1577.000000
accuracy       0.791317  0.791317  0.791317     0.791317
macro avg      0.693233  0.672136  0.681003  2027.000000
weighted avg   0.781173  0.791317  0.785302  2027.000000


In [33]:
models = {
    "normal_fatty": (normal_fatty_mlp, normal_fatty_std, normal_fatty_cols),
    "normal_cirrhosis": (normal_cirrhosis_mlp, normal_cirrhosis_std, normal_cirrhosis_cols),
    "fatty_cirrhosis": (fatty_cirrhosis_mlp, fatty_cirrhosis_std, fatty_cirrhosis_cols)
}

X_test = test_data.copy()
y_test = X_test.pop('target')

In [34]:
predictions = {}
for name in models.keys():
    cols = models[name][2]
    X_test = test_data.copy()
    y_test = X_test.pop('target')
    X_test = pd.DataFrame(models[name][1].transform(X_test[cols]), columns = cols, index = X_test.index)
    X_test =  X_test[cols]
    y_pred = pd.Series(models[name][0].predict(X_test),index=y_test.index)
    predictions[name] = images_pred(y_pred)
    
image_names = np.unique(y_test.index)

In [35]:
image_name = np.unique(y_test.index)
final_pred = {}
for image in image_name:
    pred = {
        'normal': 0,
        'fatty': 0,
        'cirrhosis': 0
    }
    for model in predictions.keys():
        pred[predictions[model][image]] += 1
    cls = max(pred, key=pred.get)
    if pred[cls] == 1:
        final_pred[image] = 'abstain'
    else: final_pred[image] = cls

In [36]:
images_acc(y_test, final_pred)

0.625

In [37]:
y_test = y_test[~y_test.index.duplicated(keep='first')].sort_index()
y_pred = pd.Series(final_pred).sort_index()
abstain = y_pred[y_pred=='abstain'].index

y_test = y_test.drop(abstain)
y_pred = y_pred.drop(abstain)

report = classification_report(y_test, y_pred, output_dict = True)
cr = pd.DataFrame(report).transpose()
print(cr)
print("Abstention Rate: ", len(abstain)/(len(y_pred)+len(abstain)))

              precision    recall  f1-score    support
cirrhosis      0.142857  0.125000  0.133333   8.000000
fatty          0.734375  0.921569  0.817391  51.000000
normal         1.000000  0.142857  0.250000  14.000000
accuracy       0.684932  0.684932  0.684932   0.684932
macro avg      0.625744  0.396475  0.400242  73.000000
weighted avg   0.720493  0.684932  0.633611  73.000000
Abstention Rate:  0.0875
